In [1]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 81.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import time
import torch.utils.data
import os
import sys
import rasterio
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import random
import re
import numpy
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.metrics import confusion_matrix, f1_score, jaccard_score
import seaborn as sns
import pandas as pd

from concurrent.futures import ThreadPoolExecutor, as_completed


import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import torch

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF
import random
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torchvision.transforms as T
from torchvision.transforms import InterpolationMode
import random


import torchvision.transforms as T
from torch.utils.data import DataLoader
from torchvision.transforms import InterpolationMode
from torchvision.transforms.functional import to_tensor
import random
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from math import cos,pi
from sklearn.metrics import f1_score, precision_score, recall_score, jaccard_score, accuracy_score, confusion_matrix
from scipy.ndimage import morphology
from scipy.ndimage.filters import maximum_filter1d
from torch.nn import Module, Sequential
from torch.nn import Conv3d, ConvTranspose3d, BatchNorm3d, MaxPool3d, AvgPool1d, Dropout3d
from torch.nn import ReLU, Sigmoid
from torch.utils.data import DataLoader
from torch.autograd import Variable

import os
import rasterio
from glob import glob
import matplotlib.pyplot as plt
from rasterio.plot import show
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import seaborn as sns
import pandas as pd
import math

import os
import numpy as np
import torch
import rasterio
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter

from torch.utils.data import Dataset
from torch.utils.data import random_split

from torch.utils.data import ConcatDataset, DataLoader

from scipy.stats import mode

import os
import numpy as np
import tifffile
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import numpy as np
from scipy.stats import skew, kurtosis
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

/tmp/ipykernel_36/2216408665.py:59: DeprecationWarning: Please import `maximum_filter1d` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import maximum_filter1d


In [5]:
BASE_PATH = '/kaggle/input/sentinel2-crop-mapping'
FOLDERS = ['lombardia', 'lombardia2']
YEARS = ['data2016', 'data2017', 'data2018']
BANDS = 9
IMG_SIZE = (48, 48)
MAX_TILES = 50  # change as needed

label_remap = {
    2: 0, 9: 1, 12: 2, 7: 3,
    1: 4, 3: 4, 5: 4, 6: 4, 8: 4, 10: 4, 11: 4, 13: 4, 14: 4, 15: 4, 16: 4, 19: 4, 255: 4,
    4: 5,
    17: 6, 18: 6, 20: 6, 21: 6
}

def remap_labels(mask):
    return np.vectorize(lambda x: label_remap.get(x, 255))(mask).astype(np.uint8)

def is_valid_image(file):
    return file.endswith('.tif') and '_MSAVI' not in file and file != 'y.tif'

all_pixel_metadata = []  # (tile_path, i, j)
all_band_values = [[] for _ in range(BANDS)]  # for histograms & stats
tot_pix_count =0 

print("🔍 Scanning tiles...")
for region in FOLDERS:
    for year in YEARS:
        root = os.path.join(BASE_PATH, region, year)
        tile_ids = sorted(os.listdir(root))[:MAX_TILES]
        
        for tile_id in tqdm(tile_ids, desc=f"{region}/{year}"):
            tile_path = os.path.join(root, tile_id)
            try:
                # Load label
                with rasterio.open(os.path.join(tile_path, 'y.tif')) as src:
                    y_mask = remap_labels(src.read(1))  # (48,48)

                # Collect valid pixels
                for i in range(48):
                    for j in range(48):
                        if y_mask[i, j] != 255:
                            all_pixel_metadata.append((tile_path, i, j))
                            tot_pix_count += 1
            except:
                continue

print("Total Pixels: ",tot_pix_count)

🔍 Scanning tiles...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/sentinel2-crop-mapping/lombardia/data2016'

In [ ]:
label_remap = {
    2: 0,  # Cereals
    9: 1,  # Maize
    12: 2, # Rice
    7: 3,  # Forage
    1: 4, 3: 4, 5: 4, 6: 4, 8: 4, 10: 4, 11: 4, 13: 4, 14: 4, 15: 4, 16: 4, 19: 4, 255:4,  # Unknown crop
    4: 5,  # Woods/tree crops
    17: 6, 18: 6, 20: 6, 21: 6,  # Non-agricultural


de remap_labels(mask, label_remap):
    return np.vectorize(lambda x: label_remap.get(x, 255))(mask).astype(np.uint8)


In [ ]:


def plot_band_histogram(X, band=0, timestep=0, title_prefix=""):

    band_data = X[timestep, band].flatten()
    
    plt.figure(figsize=(7, 4))
    plt.hist(band_data, bins=100, color='blue', alpha=0.7)
    plt.title(f"{title_prefix} Band {band} Histogram @ t={timestep}")
    plt.xlabel("Pixel Value")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.tight_layout()
    plt.show()






In [ ]:

band_sums = np.zeros(BANDS, dtype=np.float64)
band_sqsums = np.zeros(BANDS, dtype=np.float64)
band_counts = np.zeros(BANDS, dtype=np.int64)
all_data = []


for region in FOLDERS:
    for year in YEARS:
        tiles_root = os.path.join(BASE_PATH, region, year)
        tile_ids = sorted(os.listdir(tiles_root))
        tile_paths = [os.path.join(tiles_root, tile_id) for tile_id in tile_ids[:2500:10]]

        with ThreadPoolExecutor(max_workers=32) as executor:
            futures = [executor.submit(process_tile, path) for path in tile_paths]
            for f in tqdm(as_completed(futures), total=len(futures), desc=f"{region}/{year}"):
                result = f.result()
                if result is not None:
                    all_data.append(result)


# mean and std
means = band_sums / band_counts
stds = np.sqrt(band_sqsums / band_counts - means ** 2)
print("\n📊 Per-band means:", np.round(means, 2))
print("📊 Per-band stds: ", np.round(stds, 2))




In [ ]:


def print_band_summary(X, timestep=0, sample_id=0):
    print(f"\n📦 Sample {sample_id}, Timestep {timestep}, Shape: {X.shape}")
    print("Band | Mean     Std     Min     Max     P25     P50     P75     Skew   Kurt")

    for b in range(X.shape[1]):
        band_data = X[timestep, b].astype(np.float64).flatten()
        mean = band_data.mean()
        std = band_data.std()
        vmin = band_data.min()
        vmax = band_data.max()
        p25 = np.percentile(band_data, 25)
        p50 = np.percentile(band_data, 50)
        p75 = np.percentile(band_data, 75)
        sk = skew(band_data)
        kurt = kurtosis(band_data)
        
        print(f"{b:>4} | {mean:7.2f} {std:7.2f} {vmin:7.2f} {vmax:7.2f} {p25:7.2f} {p50:7.2f} {p75:7.2f} {sk:7.2f} {kurt:7.2f}")

# Example usage:
X, y = all_data[0]
print_band_summary(X, timestep=0, sample_id=0)

for b in range(9):  # for all 9 bands
    X, _ = all_data[13]
    plot_band_histogram(X, band=b, timestep=0, title_prefix="Before Norm")

In [ ]:
#Z-score normalisaiton
for i in range(len(all_data)):
    X, y = all_data[i]
    X = X.astype(np.float32)  
    for b in range(BANDS):
        X[:, b] = (X[:, b] - means[b]) / stds[b]
    all_data[i] = (X, y)


In [ ]:

def print_band_summary(X, timestep=0, sample_id=0):
    print(f"\n📦 Sample {sample_id}, Timestep {timestep}, Shape: {X.shape}")
    print("Band | Mean     Std     Min     Max     P25     P50     P75     Skew   Kurt")

    for b in range(X.shape[1]):
        band_data = X[timestep, b].astype(np.float64).flatten()
        mean = band_data.mean()
        std = band_data.std()
        vmin = band_data.min()
        vmax = band_data.max()
        p25 = np.percentile(band_data, 25)
        p50 = np.percentile(band_data, 50)
        p75 = np.percentile(band_data, 75)
        sk = skew(band_data)
        kurt = kurtosis(band_data)
        
        print(f"{b:>4} | {mean:7.2f} {std:7.2f} {vmin:7.2f} {vmax:7.2f} {p25:7.2f} {p50:7.2f} {p75:7.2f} {sk:7.2f} {kurt:7.2f}")

# Example usage:
X, y = all_data[0]
print_band_summary(X, timestep=0, sample_id=0)

for b in range(9):  # for all 9 bands
    X, _ = all_data[13]
    plot_band_histogram(X, band=b, timestep=0, title_prefix="After Norm")

In [ ]:
label_counter = Counter()

for _, y in all_data:
    flat_labels = y.flatten()
    label_counter.update(flat_labels.tolist())

# Sort the label counts
label_counts = dict(sorted(label_counter.items()))

# ✅ Total number of unique labels
num_labels = len(label_counts)

# ✅ Total number of labeled pixels
total_pixels = sum(label_counts.values())



# 🔍 Print everything
print(f"🔢 Total unique labels: {num_labels}")
print(f"🧮 Total labeled pixels: {total_pixels}")
print("📊 Label Frequencies:")
for label, count in label_counts.items():
    print(f"  Label {label}: {count} pixels")

In [ ]:

label_counter = Counter()

for _, mask in all_data:
    unique, counts = np.unique(mask, return_counts=True)
    label_counter.update(dict(zip(unique, counts)))

print("Class pixel counts:", dict(label_counter))

num_classes = 7
total_pixels = sum(label_counter.values())

weights = []
for cls in range(num_classes):
    cls_count = label_counter.get(cls, 0)
    if cls_count == 0:
        weights.append(0.0)  # Unused class
    else:
        freq = cls_count / total_pixels
        weights.append(1.0 / freq)

# Step 5: Convert to torch.Tensor
class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

print("Class weights:", class_weights)


In [ ]:
#fixed seed
np.random.seed(13)

#80/20 split
total_len = len(all_data)
train_len = int(0.8 * total_len)
val_len = total_len - train_len
train_raw, val_raw = random_split(all_data, [train_len, val_len])

print(f"Total samples: {len(all_data)}")
print(f"Train samples: {len(train_raw)}")
print(f"Test samples:  {len(val_raw)}")

In [ ]:

def analyze_timesteps(data):
    timestep_counts = defaultdict(int)

    for X, _ in data:
        T = X.shape[0]
        timestep_counts[T] += 1

    # Print detailed info
    print("📊 Unique Time Step Counts:")
    for T in sorted(timestep_counts):
        print(f"  T = {T}: {timestep_counts[T]} tiles")

    print(f"\n🧮 Total Unique T values: {len(timestep_counts)}")
    return timestep_counts

# Example usage
analyze_timesteps(train_raw)



In [ ]:
def plot_sample(X, y, timestep=0, band=3):
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    img = X[timestep, band]
    
    # Normalize visualization range to make contrast visible
    vmin = np.percentile(img, 2)
    vmax = np.percentile(img, 98)

    axs[0].imshow(img, cmap='gray', vmin=vmin, vmax=vmax)
    axs[0].set_title(f'Band {band} @ t={timestep}')

    axs[1].imshow(y, cmap='tab20')
    axs[1].set_title('Ground Truth')

    plt.tight_layout()
    plt.show()


# Plot a sample
sample_X, sample_y = all_data[20]
print(f"Image sequence shape: {sample_X.shape}")  # (T, 9, 48, 48)
print(f"Ground truth shape: {sample_y.shape}")    # (48, 48)
plot_sample(sample_X, sample_y, timestep=0, band=2)


# Model

In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels=32*9, num_classes=7, dropout=0.3):
        super().__init__()

        def res_block(in_ch, out_ch, dilation):
            skip = nn.Conv2d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 3, padding=dilation, dilation=dilation),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(),
                nn.Dropout2d(dropout),
                
                nn.Conv2d(out_ch, out_ch, 3, padding=dilation, dilation=dilation),
                nn.BatchNorm2d(out_ch),
                nn.Dropout2d(dropout),
                nn.ReLU(),
                
                nn.Sequential(skip)  # identity if same shape, otherwise 1x1 conv
            )

        self.encoder1 = res_block(in_channels, 256, dilation=1)
        self.encoder2 = res_block(256, 256, dilation=2)
        self.encoder3 = res_block(256, 256, dilation=4)
        self.encoder4 = res_block(256, 512, dilation=2)
        self.encoder5 = res_block(512, 512, dilation=1)
        self.final_conv = nn.Conv2d(512, num_classes, kernel_size=1)

    def forward(self, x):
        # Manual forward pass for residual + skip connections
        def forward_res_block(x, block):
            skip = block[-1][0](x)
            out = block[0](x)
            out = block[1](out)
            out = block[2](out)
            out = block[3](out)
            out = block[4](out)
            out = block[5](out)
            out = block[6](out)
            return F.relu(out + skip)

        x = forward_res_block(x, self.encoder1)
        x = forward_res_block(x, self.encoder2)
        x = forward_res_block(x, self.encoder3)
        x = forward_res_block(x, self.encoder4)
        x = forward_res_block(x, self.encoder5)
        return self.final_conv(x)


In [ ]:

class CustomDataset(Dataset):
    def __init__(self, tiles, label_smoothing=0.1, num_classes=7, augment=False, noise_std=0.01):
        self.tiles = tiles
        self.augment = augment
        self.label_smoothing = label_smoothing
        self.num_classes = num_classes
        self.noise_std = noise_std

        self.aug = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.RandomRotation(15, interpolation=InterpolationMode.BILINEAR),
        ])

    def __len__(self):
        return len(self.tiles)

    def __getitem__(self, idx):
        X, y = self.tiles[idx]  # X: (32, 9, 48, 48), y: (48, 48)
        X = torch.from_numpy(X.reshape(-1, 48, 48)).clone()  # shape: (T*9, 48, 48)
        y = torch.from_numpy(y).long().clone()    # (32*9, 48, 48)

        if self.augment:
            seed = random.randint(0, 99999)
            torch.manual_seed(seed)
            X = self.aug(X)
            torch.manual_seed(seed)
            y = self.aug(y.unsqueeze(0)).squeeze(0).long()

        #gaussian noise
        X = X + torch.randn_like(X) * self.noise_std

        return X.float(), y.long()  

def get_dataloaders(train_tiles, val_tiles, batch_size=16, timesteps=32):
    train_tiles = [(X[:timesteps], y) for X, y in train_tiles]
    val_tiles = [(X[:timesteps], y) for X, y in val_tiles]

    train_ds = CustomDataset(train_tiles, augment=True)
    val_ds = CustomDataset(val_tiles, augment=False)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
    return train_loader, val_loader



# Training

In [ ]:
t=32
num_epochs = 30
lr = 1e-3
weight_decay = 1e-4
label_smoothing = 0.1

train_loader, val_loader = get_dataloaders(train_raw, val_raw, batch_size=16, timesteps=t)

model = CNN(in_channels=t*9, num_classes=7).to(device) 
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
criterion = nn.CrossEntropyLoss(weight = class_weights,label_smoothing=label_smoothing)

train_losses, val_losses = [], []
train_accs, val_accs = [], []

# --- Training Loop ---
for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for X, y in tqdm(train_loader, desc=f"[Train] Epoch {epoch+1}/{num_epochs}"):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = logits.argmax(1)
        correct += (pred == y).sum().item()
        total += y.numel()

    train_losses.append(total_loss / len(train_loader))
    train_accs.append(100 * correct / total)

    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for X, y in tqdm(val_loader, desc=f"[Val] Epoch {epoch+1}/{num_epochs}", leave=False):
            X, y = X.to(device), y.to(device)
            logits = model(X)
            loss = criterion(logits, y)
            val_loss += loss.item()
            pred = logits.argmax(1)
            val_correct += (pred == y).sum().item()
            val_total += y.numel()

    val_losses.append(val_loss / len(val_loader))
    val_accs.append(100 * val_correct / val_total)
    scheduler.step()

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_losses[-1]:.4f} | Val Loss: {val_losses[-1]:.4f} | "
          f"Train Acc: {train_accs[-1]:.2f}% | Val Acc: {val_accs[-1]:.2f}%")




In [ ]:
# --- Plotting ---
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Val Loss")
plt.title("Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label="Train Acc")
plt.plot(val_accs, label="Val Acc")
plt.title("Accuracy")
plt.legend()

plt.tight_layout()
plt.show()

# Evalutaion

In [ ]:
def visualize(model, dataset, idx=0, num_classes=7, label_names=None, rgb_bands=(2, 1, 0), timestep=16):
    model.eval()

    # === Load data ===
    X, y = dataset[idx]           # X: (C*T, H, W), y: (H, W)
    X_input = X.unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(X_input).argmax(1).squeeze().cpu().numpy()

    y = y.numpy()

    # === Reshape for RGB ===
    C = 9
    T = X.shape[0] // C
    H, W = X.shape[1], X.shape[2]

    X_vis = X.view(T, C, H, W)  # (T, C, H, W)
    rgb = X_vis[timestep][list(rgb_bands)].numpy()
    rgb = np.transpose(rgb, (1, 2, 0))
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-6)  # normalize

    # === Color Map and Legend ===
    cmap = plt.cm.get_cmap('tab10', num_classes)
    pred_classes = np.unique(pred)
    gt_classes = np.unique(y)

    pred_legend = [mpatches.Patch(color=cmap(cls), label=label_names[cls]) for cls in pred_classes] if label_names else []
    gt_legend = [mpatches.Patch(color=cmap(cls), label=label_names[cls]) for cls in gt_classes] if label_names else []

    # === Plotting ===
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))

    # Top row: Pred and GT
    axs[0, 0].imshow(pred, cmap=cmap, vmin=0, vmax=num_classes-1)
    axs[0, 0].set_title("Predicted")
    axs[0, 0].axis("off")
    if pred_legend:
        axs[0, 0].legend(handles=pred_legend, bbox_to_anchor=(1.05, 1), loc='upper left')

    axs[0, 1].imshow(y, cmap=cmap, vmin=0, vmax=num_classes-1)
    axs[0, 1].set_title("Ground Truth")
    axs[0, 1].axis("off")
    if gt_legend:
        axs[0, 1].legend(handles=gt_legend, bbox_to_anchor=(1.05, 1), loc='upper left')

    # Bottom left: Satellite RGB
    axs[1, 0].imshow(rgb)
    axs[1, 0].set_title(f"Satellite RGB @ timestep {timestep}")
    axs[1, 0].axis("off")

    # Bottom right: hide unused
    axs[1, 1].axis("off")

    plt.tight_layout()
    plt.show()

label_names = {
    0: "Cereals",
    1: "Maize",
    2: "Rice",
    3: "Forage",
    4: "Unknown crop",
    5: "Woods/tree crops",
    6: "Non-agricultural"
}

val_tile = [(X[:32], y) for X, y in val_raw]
val_ds = CustomDataset(val_tile, augment=False)
visualize(model, val_ds, idx=15, num_classes=7, label_names=label_names,timestep = 16)





In [ ]:
def evaluate_combined(model, loader, label_names, num_classes=7):
    model.eval()
    pred_counts = np.zeros(num_classes, dtype=int)
    gt_counts = np.zeros(num_classes, dtype=int)
    correct = 0
    total = 0

    all_preds = []
    all_gts = []

    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            preds = model(X).argmax(1)

            preds_np = preds.cpu().numpy().flatten()
            y_np = y.cpu().numpy().flatten()

            for cls in range(num_classes):
                pred_counts[cls] += np.sum(preds_np == cls)
                gt_counts[cls] += np.sum(y_np == cls)

            correct += np.sum(preds_np == y_np)
            total += y_np.size

            all_preds.extend(preds_np)
            all_gts.extend(y_np)

    all_preds = np.array(all_preds)
    all_gts = np.array(all_gts)

    acc = correct / total

    cm = confusion_matrix(all_gts, all_preds, labels=list(range(num_classes)))
    intersection = np.diag(cm)
    union = np.sum(cm, axis=0) + np.sum(cm, axis=1) - intersection
    iou = np.round(intersection / (union + 1e-10),4)
    class_acc = np.round(intersection / (np.sum(cm, axis=1) + 1e-10),4)

    df = pd.DataFrame({
        'Class': [label_names[c] for c in range(num_classes)],
        'Total Pixels': gt_counts,
        'Pred Pixels': pred_counts,
        'Accuracy': class_acc,
        'IoU': iou
    })

    print(df.to_string(index=False))
    print(f"\nTotal Accuracy: {acc:.4f}")
    print(f"Mean IoU: {np.mean(iou):.4f}")
    print(f"F1 Score (macro): {f1_score(all_gts, all_preds, average='macro'):.4f}")

    plt.figure(figsize=(10, 5))
    bar_width = 0.35
    x = np.arange(num_classes)
    plt.bar(x - bar_width/2, gt_counts, bar_width, label='Ground Truth', color='skyblue')
    plt.bar(x + bar_width/2, pred_counts, bar_width, label='Predicted', color='salmon')
    plt.xlabel("Classes")
    plt.ylabel("Pixel Count")
    plt.title("Pixel Distribution per Class")
    plt.xticks(x, [label_names[cls] for cls in range(num_classes)], rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=[label_names[i] for i in range(num_classes)],
                yticklabels=[label_names[i] for i in range(num_classes)], cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.show()


evaluate_combined(model, val_loader, label_names, num_classes=7)

# Loading Model

In [ ]:
epoch = 10
model.load_state_dict(torch.load(f"model_epoch_{epoch}.pth"))
model.to(device)
